In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

## Dinamika hitrosti vozila

Ta interaktivni primer prikazuje popis dinamike hitrosti vozila z uporabo diferencialnih enačb ter pretvorbo le-teh v prostor stanj.

Dinamika hitrosti vozila je pod vplivom dveh glavnih sil: zdrsa in zračnega upora. Podatki za konkreten primer tega sistema so: masa vozila $m = 1000$ kg, maximalni navor motorja $\tau_{\text{max}}=150$ Nm, premer koles $r=25$ cm (poenostavimo, da je prestavno razmerje $\eta$ enako 4:1) ter koeficient zračnega upora  $b = 60$ Ns/m (zračni upor je modeliran kot linearna funkcija hitrosti vozila $v$). 

Predpostavimo, da lahko dinamiko motorja, od trenutka pritiska na stopalko za plin do dosega želenega deleža maksimalnega navora $\tau_{\%}$, modeliramo na naslednji način:

$$
\dot{\tau}_{\%}=-\frac{1}{2}\tau_{\%}+\frac{1}{2}t_{r}
$$

Iz zgornjega modela vidimo, da gre za linearni sistem prvega reda s časovno konstanto $T=2$ s in enotskim ojačanjem.

### Diferencialna enačba sistema
Na podlagi drugega Newtonovega zakona ($F=ma$) lahko zapišemo diferencialno enačbo, ki popisuje gibanje vozila:

$$
m\dot{v}= \frac{\tau_{\text{max}}\eta}{r}\tau_{\%}-bv,
$$

kjer dejanski navor, ki ga generira motor, dobimo z zmnožkom $\tau_{\%}$ in maksimalnega navora $\tau_{\text{max}}$. Odziv sistema lahko nato opišemo z naslednjim sistemom diferencialnih enačb:

$$
\begin{cases}
m\dot{v}= \frac{\tau_{\text{max}}\eta}{r}\tau_{\%}-bv \\
\dot{\tau}_{\%}=-\frac{1}{2}\tau_{\%}+\frac{1}{2}t_{r}
\end{cases}
$$


### Sistem v prostoru stanj

Zgornji diferencialni enačbi sta prvega reda, tako da je število potrebnih stanj, s katerim lahko popišemo delovanje sistema, enako 2. Če definiramo vektor stanj $x=\begin{bmatrix}x_1&x_2\end{bmatrix}^T=\begin{bmatrix}v&\tau_{\%}\end{bmatrix}^T$ in obravnavamo $t_r$ kot vhod v sistem, lahko zapišemo sistem diferencialnih enačb v prostoru stanj kot:

$$
\dot{x}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}\\0&-\frac{1}{2}\end{bmatrix}}_{A}x+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\end{bmatrix}}_{B}t_r \\
$$

### Kako upravljati s tem interaktivnim primerom?

Spreminjaj parametre sistema in poizkusi odgovoriti na naslednja vprašanja:
- Ali lahko obravnavan, poenostavljen model vozila doseže neskončno hitrost? Zakaj?
- Ali je maksimalna hitrost vozila odvisna od mase vozila? Zakaj?
- Ali je negativna vrednost $t_r$ relevantna v obravnavanem modelu? Zakaj?

In [2]:
#Preparatory Cell 

%matplotlib notebook
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline
#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define matrixes
C = numpy.matrix([[1,0],[0,1]])
D = numpy.matrix([[0],[0]])
X0 = matrixWidget(2,1)


m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='m [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
eta = widgets.FloatSlider(
    value=4,
    min=0.8,
    max=10.0,
    step=0.1,
    description='$\eta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
tau_max = widgets.FloatSlider(
    value=150,
    min=50,
    max=900,
    step=1,
    description=r'$\tau_{\text{max}}$ [Nm]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
b_air = widgets.FloatSlider(
    value=60,
    min=0,
    max=200,
    step=1,
    description=r'$b$ [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
u = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
omega = widgets.FloatSlider(
    value=5,
    min=0,
    max=10.0,
    step=0.1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Pritisni!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Pritisni za spremembo začetnih pogojev',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

#define type of ipout 
SELECT = widgets.Dropdown(
    options=['impulzna funkcija', 'koračna funkcija', 'sinusoidna funkcija'],
    value='koračna funkcija',
    description='',
    disabled=False
)

In [5]:
def main_callback(X0, m, eta, tau_max, b_air, u, selu, omega, DW):
    r = 0.25 # m
    a = numpy.matrix([[-b_air/m,tau_max*eta/m/r],[0,-1/2]])
    b = numpy.matrix([[0],[1/2]])

    eig = numpy.linalg.eig(a)
    sys = sss(a,b,C,D)
    
    if min(numpy.real(abs(eig[0]))) != 0:
        T = numpy.linspace(0,10/min(numpy.real(abs(eig[0]))),1000)
    else:
        if max(numpy.real(abs(eig[0]))) != 0:
            T = numpy.linspace(0,10/max(numpy.real(abs(eig[0]))),1000)
        else:
            T = numpy.linspace(0,180,1000)
        
    if selu == 'impulzna funkcija': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        y = control.forced_response(sys,T,U,X0)
    if selu == 'koračna funkcija':
        U = [u for t in range(0,len(T))]
        y = control.forced_response(sys,T,U,X0)
    if selu == 'sinusoidna funkcija':
        U = u*numpy.sin(omega*T)
        y = control.forced_response(sys,T,U,X0)
    
    fig=plt.figure(num=1,figsize=[15, 4])
    fig.add_subplot(121)
    plt.plot(T,y[1][0])
    plt.grid()
    plt.xlabel('čas [s]')
    plt.ylabel('hitrost [m/s]')
    
    fig.add_subplot(122)
    plt.plot(T,y[1][1])
    plt.grid()
    plt.xlabel('čas [s]')
    plt.ylabel(r'$\tau_\%$')
    
    #display(Markdown('The A matrix is: $%s$ and the eigenvalues are: $%s$' % (bmatrix(a),eig[0])))

#create a graphic structure to hold all widgets 
alltogether =  widgets.VBox([widgets.HBox([widgets.VBox([m,
                                           eta,
                                           tau_max,
                                           b_air]),
                            widgets.HBox([widgets.VBox([widgets.Label('Izberi vhodno funkcijo:',border=3),
                                                        widgets.Label('$t_r$:',border=3),
                                                        widgets.Label('omega [rad/s]:',border=3)]),
                                          widgets.VBox([SELECT,u,omega])])]),
                             widgets.HBox([widgets.Label('Začetni pogoji X0:',border=3),X0,
                                           widgets.Label('Pritisni za spremembo začetnih pogojev:',border=3),START])])

out = widgets.interactive_output(main_callback,{'X0':X0, 'm': m, 'eta': eta, 'tau_max': tau_max, 'b_air': b_air, 'u': u, 'selu': SELECT, 'omega':omega, 'DW':DW})
#out.layout.height = '200px'
display(out,alltogether)

Output()

In [6]:
#create dummy widget 2
DW2 = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))
DW2.value = -1

#create button widget
START2 = widgets.Button(
    description='Prikaži pravilne odgovore',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Pritisni za preverbo odgovorov',
    icon='check',
    layout=widgets.Layout(width='200px', height='auto')
)
                       
def on_start_button_clicked2(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW2.value> 0 :
        DW2.value = -1
    else: 
        DW2.value = 1
    pass
START2.on_click(on_start_button_clicked2)

def main_callback2(DW2):
    if DW2 > 0:
        display(Markdown(r'''>Odgovori:
                            >- Da, a le če je koeficient zračnega upora enak 0.
                            >- Ne, masa avtomobila vpliva le na čas, potreben, da se doseže določeno hitrost; vpliva torej na pospešek vozila in ne na njegovo maksimalno hitrost. 
                            >- Da, lahko se uporabi za modeliranje procesa zaviranja vozila.'''))
    else:
        display(Markdown(''))

#create a graphic structure to hold all widgets 
alltogether2 =  widgets.VBox([START2])

out2 = widgets.interactive_output(main_callback2,{'DW2':DW2})
#out.layout.height = '300px'
display(out2,alltogether2)

Output()